In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Question 4.1

In [3]:
data = pd.read_csv("/Users/stephenandrews/Downloads/MIT Schoolwork/Spring 2025/6.3732/6373Pset4/data/Cooffending_data.csv")

nparr = data.to_numpy()

print(data)

         NoUnique            Name  Naissance SEXE     SeqE   dateInf   NCD1  \
0               1        Tatielly       1007    F  1085034  20051217   3530   
1               2       Oluwasayo       1828    F  1431379  20080423   1430   
2               4    ناصرعلي محمد       1889    M   167174  20080306   1430   
3               5              炜杰       1892    M  1179096  20080821   1420   
4              17           Avare       1897    M  1270690  20030430   1625   
...           ...             ...        ...  ...      ...       ...    ...   
1280454    670532         Liriane       2004    F  1112491  20090326   1330   
1280455    670533        Benzacky       2004    M    78081  20090629  21702   
1280456    670534      Piscitelli       2005    M   187904  20090316  21201   
1280457    670535  Alvaro Ramirez       2005    M   523837  20091018  21704   
1280458    670536            Kbir       2005    M  1632652  20081124  21101   

          NCD2 NCD3 NCD4    MUN ED1  Jeunes  Adulte

#### Part (a)

In [4]:
len(np.unique(nparr[:, 0]))

539593

#### Part (b)

In [27]:
len(np.unique(nparr[:, 1]))

539593

#### Part (c)

In [26]:
print(f"All years: {len(np.unique(nparr[:, 4]))}")

years = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010]

for year in years:
    arr = np.where(nparr[:, -1] == year)
    # print(nparr[arr][:, 4])
    num = len(np.unique(nparr[arr][:, 4]))
    print(f"{year}: {num}")


All years: 1164836
2003: 110556
2004: 121374
2005: 172664
2006: 185840
2007: 196151
2008: 199288
2009: 178959
2010: 4


#### part (d)

In [49]:
unique_crimes = np.unique(nparr[:, 4])

unique_vals, counts = np.unique(nparr[:, 4], return_counts=True)

sorted_indices = np.argsort(-counts)

sorted_unique_vals = unique_vals[sorted_indices]
sorted_counts = counts[sorted_indices]

result = np.column_stack((sorted_unique_vals, sorted_counts))

In [48]:
muns = np.zeros((10, 1))

count = 0

for i in result[:10]:
    idx = np.where(nparr[:, 4] == i[0])[0][0]

    muns[count] = nparr[idx][10]
    count += 1

print(np.column_stack((sorted_unique_vals[:10], sorted_counts[:10], muns)))

[[27849 156 66023.0]
 [876159 102 12072.0]
 [445040 77 66023.0]
 [23526 60 75017.0]
 [60815 60 54048.0]
 [212285 57 37067.0]
 [754187 56 95032.0]
 [539385 52 66023.0]
 [23610 51 78005.0]
 [453024 46 66023.0]]
